## Script para gerar as listas dos sabores partilhados entre os ingredientes de cada receita válida
- Listar os ingredientes encontrados através de matching_flavorNetwork_ptDataset
- Eliminar da flavorNetwork(ingr_comp) os ingredientes que não fazem parte dessa lista
- Para cada um dos ingredientes listar os seus componentes de sabor (compound id) https://stackoverflow.com/questions/22219004/how-to-group-dataframe-rows-into-list-in-pandas-groupby
- Para cada um dos ingredientes comparar a sua lista de componenenetes de sabor com a lista dos restantes ingredientes, ver se partilham e contar a ocorrencias
- Gerar uma lista para todos os pares de ingredientes e a quantidade de componenetes de sabor que partilham 
- Gerar uma lista para cada receita com todos os pares de ingredientes e a quantidade de componenetes de sabor que partilham

In [1]:
import pandas as pd
import itertools
import re
import numpy as np
import igraph as ig
from scipy.spatial.distance import squareform
from sortedcontainers import SortedDict
import math

#csv para dataframe
#limpar/apagar colunas/linhas indesejadas
#criação dos dataframe originais

matching_flavorNetwork_ptDataset_df = pd.read_csv("matching_flavorNetwork_ptDataset.csv")
matching_flavorNetwork_ptDataset_df = matching_flavorNetwork_ptDataset_df.drop(['ing_pt','ing_eng'], axis=1)
matching_flavorNetwork_ptDataset_df = matching_flavorNetwork_ptDataset_df[:-3]
flavor_network_ing_comp_df = pd.read_csv('flavor_network - ingr_comp.csv')

In [2]:
#colocar numa lista todos os ingredientes encontrados no dataframe matching_flavorNetwork_ptDataset_df

found_ingredients_list = []
for i,j in matching_flavorNetwork_ptDataset_df.iterrows():
    if j['id_flavorNetwork'] != "-":
        found_ingredients_list.append(int(j['id_flavorNetwork']))

In [3]:
#criar um novo dataframe (flavor_network_valid_ing_comp_df) só com os ingredientes encontrados 

flavor_network_valid_ing_comp_df = pd.DataFrame(columns = ['# ingredient id','compound id'])
index = 0
for i,j in flavor_network_ing_comp_df.iterrows():
    if j['# ingredient id'] in found_ingredients_list:
        flavor_network_valid_ing_comp_df.loc[index] = [j['# ingredient id'],j['compound id']]
        index += 1        

In [4]:
#criar um novo dataframe (flavor_network_valid_ing_list_comp_df) agrupando os ingredientes 

flavor_network_valid_ing_list_comp_df = flavor_network_valid_ing_comp_df.groupby('# ingredient id')['compound id'].apply(list).reset_index(name='list_comp')

In [5]:
#criar um novo dataframe (flavor_network_pairwise_shared_flavors_df) comparando cada ingrediente com os outros todos e contar o número de componenetes de sabor que partilham 

flavor_network_pairwise_shared_flavors_df = pd.DataFrame(columns = ['ing 1','ing 2','shared_flavors'])
index = 0
for pair in itertools.combinations(flavor_network_valid_ing_list_comp_df['# ingredient id'], r=2):
    list1 = flavor_network_valid_ing_list_comp_df.loc[flavor_network_valid_ing_list_comp_df['# ingredient id'] == pair[0],'list_comp'].item()
    list2 = flavor_network_valid_ing_list_comp_df.loc[flavor_network_valid_ing_list_comp_df['# ingredient id'] == pair[1],'list_comp'].item()
    num_shared_flavors = len(set(list1).intersection(list2))
    flavor_network_pairwise_shared_flavors_df.loc[index] = [pair[0],pair[1],num_shared_flavors]
    index += 1

# Pipeline de operações para um dada receita

In [6]:
#carregar o csv com as receitas portuguesas
#listar os ingredientes para a receita escolhida

portuguese_dataset_valid_recipes_df = pd.read_csv("portuguese_dataset_valid_recipes.csv")
portuguese_dataset_valid_recipes_df = portuguese_dataset_valid_recipes_df.drop(['RECIPE (Portuguese)','RECIPE (English translation)','REF', 'PAGE'], axis=1)
pt_recipe_df = portuguese_dataset_valid_recipes_df.loc[portuguese_dataset_valid_recipes_df['CODE'] == 'RIBA051'].reset_index(drop=True)

ingredients_recipe_id_pt_list = []
column_number = 0
for column in pt_recipe_df.loc[0]:
    if (column == 1):
        ingredients_recipe_id_pt_list.append(re.sub(r'(?<=.{4}).+',"",pt_recipe_df.columns[column_number]))
    column_number += 1 
ingredients_recipe_id_pt_list

['I032', 'I105', 'I168', 'I224', 'I264', 'I274', 'I277', 'I292', 'I313']

In [7]:
#Listar os ingredientes da receita com o id da flavor network

ingredients_recipe_id_flavor_network_list = []
for ing in ingredients_recipe_id_pt_list:
    for i,j in matching_flavorNetwork_ptDataset_df.iterrows():
        if(j['id_pt'] == ing):
            ingredients_recipe_id_flavor_network_list.append(int(j['id_flavorNetwork']))
ingredients_recipe_id_flavor_network_list = list(dict.fromkeys(ingredients_recipe_id_flavor_network_list))
ingredients_recipe_id_flavor_network_list

[998, 1447, 1348, 215, 1113, 848, 105, 878, 1471]

In [8]:
#criar um dataframe para a receita com todos os pares de ingredientes e o número de componentes de sabor que partilham 
#criar um dataframe em forma de matriz

appended_data = []
D_pair_value = SortedDict()
recipe_df = pd.DataFrame(columns = ['ing 1','ing 2','shared_flavors'])
for pair in itertools.permutations(ingredients_recipe_id_flavor_network_list, r=2):
    data = flavor_network_pairwise_shared_flavors_df.loc[(flavor_network_pairwise_shared_flavors_df['ing 1'] == pair[0]) & (flavor_network_pairwise_shared_flavors_df['ing 2'] == pair[1])]
    appended_data.append(data)
    if (data['shared_flavors'].size):
        D_pair_value[pair] = data['shared_flavors'].values.item()
    
recipe_df = pd.concat(appended_data,ignore_index=True)
recipe_sorted_matrix_df = pd.DataFrame(squareform(D_pair_value.values()), index=ingredients_recipe_id_flavor_network_list, columns=ingredients_recipe_id_flavor_network_list)
recipe_df

,ing 1,ing 2,shared_flavors
0,998,1447,2
1,998,1348,0
2,998,1113,1
3,998,1471,0
4,1447,1471,0
5,1348,1447,13
6,1348,1471,5
7,215,998,0
8,215,1447,0
9,215,1348,0


## Script para calcular o food-pairing
- Passar o dataframe anteriror (recipe_sorted_matrix_df) para uma matriz numpy
- Média da matriz (só com valores > 0)
- Normalizar a matriz
- Aplicar a forma 1/x - 1 (incluindo os zeros)

In [9]:
recipe_np = recipe_sorted_matrix_df.to_numpy()
recipe_nan_np = recipe_np.astype('float')
recipe_nan_np[recipe_np == 0] = 'nan'
recipe_nan_np

array([[nan, nan, 15., 22.,  1., nan, 26., 10.,  1.],
       [nan, nan, nan, nan, nan, nan, nan, nan,  1.],
       [15., nan, nan, 15., nan, nan, 20., 13., nan],
       [22., nan, 15., nan, nan, nan, 25., 11.,  1.],
       [ 1., nan, nan, nan, nan,  1., nan,  2., nan],
       [nan, nan, nan, nan,  1., nan, nan,  1., nan],
       [26., nan, 20., 25., nan, nan, nan, 13.,  5.],
       [10., nan, 13., 11.,  2.,  1., 13., nan, nan],
       [ 1.,  1., nan,  1., nan, nan,  5., nan, nan]])

In [10]:
#food_pairing = np.nanmean(np.where(recipe_np!=0,recipe_np,np.nan))
#food_pairing

food_pairing = np.nanmean(recipe_nan_np)
if (math.isnan(food_pairing)):
    food_pairing = 0
food_pairing

10.166666666666666

## Script para calcular o food-bridging
- Normalizar os valores da matriz entre [0,1]
- Aplicar a fórmula 1/x -1, converte o grafo(matriz) num espaço de distância
- Aplicar Floyd-Warshall
- Dividir a matriz original pela matriz resultante da aplicação do Floyd-Warshall
- Substituir os nan por zero
- Calcular o número de valores da matriz (> 1 e < infinito)
- Calcular o número de valores da matriz (> 0 e < infinito)
- Dividir esses dois valores e o resultado é o valor de food-bridging
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.floyd_warshall.html

In [11]:
minval = np.nanmin(recipe_nan_np)
maxval = np.nanmax(recipe_nan_np)
recipe_normalized_nan_np = (((1-2*0.01)*recipe_nan_np + (2*0.01-1)*minval)/(maxval-minval))+0.01
recipe_normalized_nan_np

array([[   nan,    nan, 0.5588, 0.8332, 0.01  ,    nan, 0.99  , 0.3628,
        0.01  ],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
        0.01  ],
       [0.5588,    nan,    nan, 0.5588,    nan,    nan, 0.7548, 0.4804,
           nan],
       [0.8332,    nan, 0.5588,    nan,    nan,    nan, 0.9508, 0.402 ,
        0.01  ],
       [0.01  ,    nan,    nan,    nan,    nan, 0.01  ,    nan, 0.0492,
           nan],
       [   nan,    nan,    nan,    nan, 0.01  ,    nan,    nan, 0.01  ,
           nan],
       [0.99  ,    nan, 0.7548, 0.9508,    nan,    nan,    nan, 0.4804,
        0.1668],
       [0.3628,    nan, 0.4804, 0.402 , 0.0492, 0.01  , 0.4804,    nan,
           nan],
       [0.01  , 0.01  ,    nan, 0.01  ,    nan,    nan, 0.1668,    nan,
           nan]])

In [12]:
recipe_normalized_np = np.nan_to_num(recipe_normalized_nan_np)
recipe_normalized_form_np = (1/recipe_normalized_np) - 1
recipe_normalized_form_np

/home/ricardo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


array([[           inf,            inf, 7.89549034e-01, 2.00192031e-01,
        9.90000000e+01,            inf, 1.01010101e-02, 1.75633958e+00,
        9.90000000e+01],
       [           inf,            inf,            inf,            inf,
                   inf,            inf,            inf,            inf,
        9.90000000e+01],
       [7.89549034e-01,            inf,            inf, 7.89549034e-01,
                   inf,            inf, 3.24854266e-01, 1.08159867e+00,
                   inf],
       [2.00192031e-01,            inf, 7.89549034e-01,            inf,
                   inf,            inf, 5.17458982e-02, 1.48756219e+00,
        9.90000000e+01],
       [9.90000000e+01,            inf,            inf,            inf,
                   inf, 9.90000000e+01,            inf, 1.93252033e+01,
                   inf],
       [           inf,            inf,            inf,            inf,
        9.90000000e+01,            inf,            inf, 9.90000000e+01,
           

In [13]:
#Passar das proximidades a distancias
def prox2dist(P,k):
    D = np.power(1.0/P-1.0,k)
    return D

#Passar de distancias para proximidade
def dist2prox(D,k):
    P = 1.0/(np.power(D,1.0/k)+1.0)
    return P

#Tranformar um grafo numa matriz numpy
def graph2matrix(g):
    n = g.vcount()
    adj = np.zeros((n, n))
    rows, cols = zip(*g.get_edgelist())
    for r in rows:
        for c in cols:
            adj[r, c] = edgevalue(g,r,c)
    return adj

#Transformar uma matriz (numpy array) para um grafo
def matrix2graph(adj):
    g=ig.Graph.Weighted_Adjacency(np.ndarray.tolist(adj))
    return g

#Calcular o dijkstra, ou seja, o all pairs shortest paths num numpy array D no espaço das distancias
def all_pairs_sp(D):
    g = matrix2graph(D)
    DC = g.shortest_paths_dijkstra(weights=g.es['weight'], mode='OUT')
    DC = np.asarray(DC)
    return DC



In [14]:
A = recipe_normalized_form_np
DC = all_pairs_sp(A)
s = np.divide(A,DC)
s[np.isnan(s)] = 0
s1 = s[(s > 1) & (s < float('inf'))]
s2 = s[(s > 0) & (s < float('inf'))]
try:
    SMP = s1.size/s2.size
except ZeroDivisionError:
    SMP = 0
SMP

0.4444444444444444